In [11]:
#imports
import tkinter as tk
from tkinter import ttk
import mysql.connector
from ast import literal_eval
from tkinter.filedialog import askopenfile
import ntpath
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, 
NavigationToolbar2Tk)
import matplotlib.pyplot as plot
%matplotlib inline
import random

In [12]:
# neural network classes
# network_structure [[input_node_value],[layer[node_bias, node_value, node_weights ...]]...]
# data for calculation [input_node_1_value, input_node_2_value ...]
# data for training [[[input_node_1_value, input_node_2_value ...], [expected output1, expectout2 ...], ...]
current_neural_network = None
network_names = []
def get_neural_network_names():
    output = issue_database_command('SELECT name FROM networks')
    name_in_output = False
    for name_tuple in output:
        for name in name_tuple:
            if get_current_neural_network_name() == name:
                name_in_output = True
    if not name_in_output:
        output.append(get_current_neural_network_name())
    return output
def get_current_neural_network_name():
    if current_neural_network is None:
        return 'None'
    else:
        return current_neural_network.name
def sigmoid(input):
    return (input / (1 + abs(input))) + 0.05
    #return input
class Node:
    def __init__(self, bias, input_nodes, input_nodes_weights, value):
        self.bias = bias
        self.input_nodes = input_nodes
        self.input_nodes_weights = input_nodes_weights
        self.value = value
    def calculate_value(self):
        weighted_total = 0
        #print('num nodes: ' + str(len(self.input_nodes)))
        #print('num weights: ' + str(len(self.input_nodes_weights)))
        for weight_index, node in enumerate(self.input_nodes):
            weighted_total += float(node.value) * self.input_nodes_weights[weight_index]
        self.value = sigmoid(weighted_total + self.bias)
class Input_Node:
    def __init__(self, value):
        self.value = value
class Node_Network:
    def __init__(self, network_structure, name, variables, accuracy = 0):
        self.nodes = []
        self.name = name
        self.variables = variables
        self.accuracy = accuracy
        temp = []
        for count, layer in enumerate(network_structure):
            if count == 0:
                for value in layer:
                    temp.append(Input_Node(value))
            else:
                for node in layer:
                    bias = node[0]
                    value = node[1]
                    weights = []
                    for node_count, value in enumerate(node):
                        if node_count != 0 and node_count != 1:
                            weights.append(value)
                    temp.append(Node(bias, self.nodes[count - 1], weights, value))
            self.nodes.append(temp)
            temp = []
    def calculate(self,data):
        #print(len(data))
        #print(len(self.nodes))
        for value, node in enumerate(self.nodes[0]):
            node.value = data[value]
        for layer_count, layer in enumerate(self.nodes):
            if layer_count != 0:
                for node in layer:
                    node.calculate_value()
    def _learn(self, expected_output):
        cost = sigmoid(self._cost(expected_output))
        for layer_count, layer in enumerate(self.nodes):
            if layer_count != 0:
                for node in layer:
                    node.bias += (random.random() - 0.5) * cost
                    for weight_count, weight in enumerate(node.input_nodes_weights):
                        weight += (random.random() - 0.5) * cost
                        node.input_nodes_weights[weight_count] = weight
                            
        return
    def _cost(self, expected_output):
        output = 0
        for count, x in enumerate(expected_output):
            output += (self.nodes[len(self.nodes) - 1][count].value - float(x)) ** 2
        return output
    def get_output(self):
        output = []
        for node in self.nodes[len(self.nodes) - 1]:
            output.append(node.value)
        return output
    # input = [[headder1, header2 ...],[column1 value, column2 value ...] ...]
    # output = [[[input_node_1_value, input_node_2_value ...], [expected output 1, expect output 2 ...], ...]
    def _structure_training_data(self, data):
        data = data.data
        data.pop(0)
        for row in data:
            row.pop(0)
        output = []
        num_inputs = int(len(self.nodes[0]) / self.variables)
        for row_count, row in enumerate(data):
            if row_count >= num_inputs:
                data_inputs = []
                num_inputs_temp = num_inputs
                while num_inputs_temp > 0:
                    data_inputs.extend(data[row_count - num_inputs_temp])
                    num_inputs_temp -= 1
                full_data = []
                full_data.append(data_inputs)
                full_data.append(row)
                output.append(full_data)
        return output
    def train(self, raw_data):
        data = self._structure_training_data(raw_data)
        for dataset in data:
            self.calculate(dataset[0])
            fixed_data = []
            for index, value in enumerate(dataset[1]):
                if float(dataset[0][index]) != 0:
                    fixed_data.append(abs(float(value) / float(dataset[0][index]))) 
                else:
                    fixed_data.append(0)
            self._learn(fixed_data)
            self.update_accuracy(fixed_data)
        load_data()
    def get_structure(self):
        output = []
        for layer_count, layer in enumerate(self.nodes):
            if layer_count == 0:
                temp = []
                for node in layer:
                    temp.append(float(node.value))
                output.append(temp)
            else:
                temp_layer = []
                for node in layer:
                    temp_node = []
                    temp_node.append(node.bias)
                    temp_node.append(node.value)
                    temp_node.extend(node.input_nodes_weights)
                    temp_layer.append(temp_node)
                output.append(temp_layer)
        return output
    def update_accuracy(self, expected_output):
        output = self.get_output()
        output = output[0]
        expected_output = expected_output[0]
        if expected_output > output:
            self.accuracy = abs(output / expected_output)
        else:
            self.accuracy = abs(expected_output / output)
    def print_structure(self):
        print('Network Name: ' + str(self.name))
        for layer_count, layer in enumerate(self.nodes):
            for node_count, node in enumerate(layer):
                if layer_count > 0:
                    print('layer: ' + str(layer_count) + ' node: ' + str(node_count) + ' value: ' + str(node.value) + ' bias: ' + str(node.bias) + ' weights: ' + str(node.input_nodes_weights))
                else:
                    print('layer: ' + str(layer_count) + ' node: ' + str(node_count) + ' value: ' + str(node.value))

In [13]:
# tk window
main_window = tk.Tk()
main_window.title('Main Window')

''

In [14]:
#database stuff
database_user = 'root'
database_password = 'WGUcapstone2021'
database_host = '127.0.0.1'
database_name = 'neural_network_database'
def issue_database_command(query, commit = False):
    cnx = mysql.connector.connect(user=database_user, password=database_password,
                              host=database_host,
                              database=database_name)
    cursor = cnx.cursor()
    cursor.execute(query)
    if commit:
        cnx.commit()
    output = []
    for value in cursor:
        output.append(value)
    return output
    cnx.close()

In [15]:
#all neural network widgets
def error_pop_up(message):
    error_window = tk.Toplevel()
    error_window.title('Error!')
    error_label = tk.Label(error_window, text = message)
    error_label.pack()
def save_network():
    delete_command = "DELETE FROM networks WHERE name = '" + current_neural_network.name + "'"
    update_command = "INSERT INTO networks (name, structre, variables, accuracy) VALUES ('" + current_neural_network.name +"', '" + str(current_neural_network.get_structure()) +"', " + str(current_neural_network.variables) + ", "  + str(current_neural_network.accuracy)+ ')'
    issue_database_command(delete_command, True)
    #print(update_command)
    issue_database_command(update_command, True)
def get_current_neural_network_name():
    global current_neural_network
    if current_neural_network != None:
        return current_neural_network.name
    else:
        return 'none'
def update_network_combobox():
    change_neural_network_combobox['values'] = get_neural_network_names()
def update_network_label(event):
    if get_current_neural_network_name() != current_neural_network_string_var.get() and current_neural_network_string_var.get() != 'none':
        load_network()
        current_neural_network_string_var.set("Current Neural Network: " + get_current_neural_network_name())
neural_network_frame = tk.Frame(main_window)
current_neural_network_string_var = tk.StringVar()
current_neural_network_string_var.set("Current Neural Network: " + get_current_neural_network_name())
current_neural_network_label = tk.Label(neural_network_frame, textvariable = current_neural_network_string_var)
change_neural_network_lablel = tk.Label(neural_network_frame, text = "Change Neural Network:")
selected_network = tk.StringVar()
change_neural_network_combobox = ttk.Combobox(neural_network_frame, textvariable = selected_network, postcommand = update_network_combobox)
change_neural_network_combobox['values'] = get_neural_network_names()
change_neural_network_combobox.state(["readonly"])
change_neural_network_combobox.bind("<<ComboboxSelected>>",update_network_label)
change_neural_network_combobox.set('none')
def open_new_neural_network_window():
    create_neural_network_window = tk.Toplevel()
    create_neural_network_window.title('Create New Neural Network')
    network_name = tk.StringVar()
    network_name_label = tk.Label(create_neural_network_window, text = 'Network Name: (Must be unique)')
    input_size = tk.StringVar()
    input_size_label = tk.Label(create_neural_network_window, text = "Number of input data: (Must be int)")
    input_variables = tk.StringVar()
    input_variables_label = tk.Label(create_neural_network_window, text = 'Number of variables: (Must be int, can only be used in datasets with same number of variables)')
    network_name_entry = tk.Entry(create_neural_network_window, textvariable = network_name)
    input_size_entry = tk.Entry(create_neural_network_window, textvariable = input_size)
    input_variables_entry = tk.Entry(create_neural_network_window, textvariable = input_variables)
    additional_layers_string_vars = []
    additional_layers_entries = []
    additonal_layers_labels = []
    def add_node_layer():
        additional_layers_string_vars.append(tk.StringVar())
        additonal_layers_labels.append(tk.Label(create_neural_network_window, text = 'Layer ' + str(len(additional_layers_entries ) + 1) + ' number of nodes (Must be int)'))
        additonal_layers_labels[len(additonal_layers_labels) - 1].pack()
        additional_layers_entries.append(tk.Entry(create_neural_network_window, textvariable = additional_layers_string_vars[len(additional_layers_string_vars) - 1]))
        additional_layers_entries[len(additional_layers_entries) - 1].pack()
    def remove_node_layer():
        additional_layers_entries[len(additional_layers_entries) - 1].pack_forget()
        additonal_layers_labels[len(additonal_layers_labels) - 1].pack_forget()
        additional_layers_string_vars.pop(len(additional_layers_string_vars) - 1)
        additional_layers_entries.pop(len(additional_layers_entries) - 1)
        additonal_layers_labels.pop(len(additonal_layers_labels) - 1)
    def create_network():
        global current_neural_network
        if network_name.get() == 'none' or network_name.get() == '':
            error_pop_up('invalid network name!')
            return
        for name in get_neural_network_names():
            if get_neural_network_names() == name:
                error_pop_up('invalid network name!')
                return
        network_structure = []
        num_input_nodes = int(input_size.get()) * int(input_variables.get())
        input_values = []
        while num_input_nodes > 0:
            input_values.append(0)
            num_input_nodes -= 1
        network_structure.append(input_values)
        for count, layer in enumerate(additional_layers_string_vars):
            layer_num_nodes = int(layer.get())
            temp = []
            while layer_num_nodes > 0:
                node = [0,0]
                if count == 0:
                    previous_layer_num_nodes = int(input_size.get()) * int(input_variables.get())
                else:
                    previous_layer_num_nodes = int(additional_layers_string_vars[count - 1].get())
                while previous_layer_num_nodes > 0:
                    node.append(0)
                    previous_layer_num_nodes -= 1
                temp.append(node)
                layer_num_nodes -= 1
            network_structure.append(temp)
        num_output_nodes = int(input_variables.get())
        temp = []
        while num_output_nodes > 0:
            last_layer_num_nodes = int(additional_layers_string_vars[len(additional_layers_string_vars) - 1].get())
            output_node = [0,0]
            while last_layer_num_nodes > 0:
                output_node.append(0)
                last_layer_num_nodes -= 1
            temp.append(output_node)
            num_output_nodes -= 1
        network_structure.append(temp)
        current_neural_network = (Node_Network(network_structure, network_name.get().replace(' ', '_'), int(input_variables.get())))
        change_neural_network_combobox.set(current_neural_network.name)
        global predictions
        global num_predictions
        predictions = []
        num_predictions = 0
        create_neural_network_window.destroy()
    add_layer_button = tk.Button(create_neural_network_window, text = 'Add New Layer', command = add_node_layer)
    remove_layer_button = tk.Button(create_neural_network_window, text = 'Remove Layer', command = remove_node_layer)
    create_network_button = tk.Button(create_neural_network_window, text = 'Create Network', command = create_network)
    add_layer_button.pack()
    remove_layer_button.pack()
    create_network_button.pack()
    network_name_label.pack()
    network_name_entry.pack()
    input_size_label.pack()
    input_size_entry.pack()
    input_variables_label.pack()
    input_variables_entry.pack()
def print_current_network():
    current_neural_network.print_structure()
def load_network():
    global current_neural_network
    name = selected_network.get()
    new_network = issue_database_command("SELECT name, structre, variables, accuracy FROM networks WHERE name = '" + name + "'")
    raw_structure = new_network[0][1]
    structure = literal_eval(raw_structure)
    current_neural_network = Node_Network(structure, new_network[0][0], new_network[0][2], new_network[0][3])
    global predictions
    global num_predictions
    predictions = []
    num_predictions = 0
def train_network():
    current_neural_network.train(current_data)
def delete_network():
    delete_command = "DELETE FROM networks WHERE name ='" + current_neural_network.name + "';"
    issue_database_command(delete_command, True)
create_new_neural_network_button = tk.Button(neural_network_frame, text = 'Create new neural network', command = open_new_neural_network_window)
print_current_network_structure_button = tk.Button(neural_network_frame, text = 'Print network structure', command = print_current_network)
load_neural_network_button = tk.Button(neural_network_frame, text = 'Load selected neural network', command = load_network)
save_neural_network_button = tk.Button(neural_network_frame, text = 'Save selected neural network', command = save_network)
train_network_button = tk.Button(neural_network_frame, text= 'Train network', command = train_network)
delete_network_button = tk.Button(neural_network_frame, text = 'Delete network', command = delete_network)
predictions = []
prediction_plot_frame = tk.Frame()
pie_chart_frame = tk.Frame()
num_predictions = 0
def predict():
    global predictions
    if predictions == []:
        data_size = len(current_neural_network.nodes[0]) / int(current_neural_network.variables)
        while data_size > 0:
            new_data = current_data.data[int(len(current_data.data) - data_size)][:]
            new_data.pop(0)
            for index, data in enumerate(new_data):
                new_data[index] = float(data)
            predictions.extend(new_data)
            data_size -= 1
    # fix prediction data
    data_to_load = []
    data_size = len(current_neural_network.nodes[0])
    data_to_load = predictions[int(len(predictions) - data_size) :]
    current_neural_network.calculate(predictions)
    predictions.extend(current_neural_network.get_output())
    for index in range(current_neural_network.variables):
        predictions[len(predictions) - index - 1] = abs(current_neural_network.get_output()[index] * predictions[len(predictions) - current_neural_network.variables - index - 1])
    global prediction_plot_frame
    prediction_plot_frame.destroy()
    prediction_plot_frame = tk.Frame()
    data_to_plot = []
    for index, value in enumerate(predictions):
        if index % current_neural_network.variables == 0 and index > len(current_neural_network.nodes[0]) - 1:
            data_to_plot.append(value)
    create_bar_chart(data_to_plot, prediction_plot_frame)
    prediction_plot_frame.pack(side = 'right')
    global pie_chart_frame
    pie_chart_frame.destroy()
    pie_chart_frame = tk.Frame()
    global num_predictions
    num_predictions += 1
    adjusted_accuracy = current_neural_network.accuracy ** num_predictions
    create_pie_chart(adjusted_accuracy, pie_chart_frame)
    pie_chart_frame.pack(side = 'left')
predict_button = tk.Button(neural_network_frame, text = 'Predict', command = predict)

In [16]:
#[[headder1, header2 ...],[column1 value, column2 value ...] ...]
class Data:
    def __init__(self, name, data):
        self.name = name
        self.data = data

In [17]:
# all data widgets
current_data = None
def get_current_data_name():
    if current_data == None:
        return 'none'
    else:
        return current_data.name
def get_current_data():
    if current_data == None:
        return []
    else:
        return current_data.data
def get_data_names():
    output = issue_database_command('SELECT name FROM data')
    name_in_output = False
    for name_tuple in output:
        for name in name_tuple:
            if get_current_data_name() == name:
                name_in_output = True
    if not name_in_output:
        output.append(get_current_data_name())
    return output
def update_data_combobox():
    change_data_combobox['values'] = get_data_names()
def update_data_label(event):
    if get_current_data_name() != current_data_string_var.get() and current_data_string_var.get() != 'none':
        load_data()
        current_data_string_var.set("Current Data: " + get_current_data_name())
data_plot_frame = tk.Frame()
def load_data():
    global current_data
    name = selected_data.get()
    new_data = issue_database_command("SELECT name, data FROM data WHERE name = '" + name + "'")
    raw_data = new_data[0][1]
    data = literal_eval(raw_data)
    current_data = Data(new_data[0][0], data)
    data_to_plot = []
    for index, dataset in enumerate(current_data.data):
        if index != 0:
            data_to_plot.append(float(dataset[1]))
    global data_plot_frame
    data_plot_frame.destroy()
    data_plot_frame = tk.Frame()
    create_plot(data_to_plot, data_plot_frame)
    data_plot_frame.pack()
def save_data():
    delete_command = "DELETE FROM data WHERE name = '" + current_data.name + "'"
    update_command = "INSERT INTO data (name, data) VALUES ('" + current_data.name +"', '" + str(current_data.data).replace("'", '"') +"')"
    issue_database_command(delete_command, True)
    issue_database_command(update_command, True)
def upload_data():
    global current_data
    file_path = askopenfile(mode='r', filetypes=[('Spreadsheets', '*csv')])
    if file_path is not None:
        pass
    data = []
    for row in file_path:
        fixed_row = row.replace("\n", "")
        data.append(fixed_row.split(','))
    name = ntpath.basename(file_path.name.split('.')[0]).replace(' ', '_')
    file_path.close()
    current_data = Data(name, data)
    save_data()
    change_data_combobox.set(name)
data_frame = ttk.Frame()
current_data_string_var = tk.StringVar()
current_data_string_var.set('Current data: ' + 'None')
current_data_label = tk.Label(data_frame, textvariable = current_data_string_var)
selected_data= tk.StringVar()
change_data_combobox = ttk.Combobox(data_frame, textvariable = selected_data, postcommand = update_data_combobox)
change_data_combobox['values'] = get_data_names()
change_data_combobox.state(["readonly"])
change_data_combobox.bind("<<ComboboxSelected>>",update_data_label)
change_data_combobox.set('none')
upload_data_button = tk.Button(data_frame, text = 'Upload data', command = upload_data)
def print_data():
    print(current_data.data)
print_data_button = tk.Button(data_frame, text = 'Print data', command = print_data)
def delete_data():
    delete_command = "DELETE FROM data WHERE name ='" + current_data.name + "';"
    issue_database_command(delete_command, True)
delete_data_button = tk.Button(data_frame, text = 'Delete data', command = delete_data)

In [18]:
#matplot wdigets
visual_frame = tk.Frame()
def create_plot(data, frame):
    fig = Figure(figsize = (5, 5), dpi = 100)
    plot = fig.add_subplot(111).plot(data)
    canvas = FigureCanvasTkAgg(fig,master = frame)
    canvas.draw()
    toolbar = NavigationToolbar2Tk(canvas, frame)
    toolbar.update()
    canvas.get_tk_widget().pack()
def create_bar_chart(data, frame):
    fig = Figure(figsize = (5, 5), dpi = 100)
    plot = fig.add_subplot(111)
    x_pos = [i for i, _ in enumerate(data)]
    plot.bar(x_pos, data, color = 'blue')
    canvas = FigureCanvasTkAgg(fig,master = frame)
    canvas.draw()
    toolbar = NavigationToolbar2Tk(canvas, frame)
    toolbar.update()
    canvas.get_tk_widget().pack()
def create_pie_chart(data, frame):
    fig = Figure(figsize = (5, 5), dpi = 100)
    plot = fig.add_subplot(111)
    data *= 100
    anti_data = 100 - data
    y = [anti_data, data]
    plot.pie(y, colors = ['red', 'green'])
    canvas = FigureCanvasTkAgg(fig,master = frame)
    canvas.draw()
    toolbar = NavigationToolbar2Tk(canvas, frame)
    toolbar.update()
    canvas.get_tk_widget().pack()

In [19]:
#pack all widgets
neural_network_frame.pack(side = 'left')
data_frame.pack(side = 'right')
current_neural_network_label.pack()
change_neural_network_lablel.pack()
change_neural_network_combobox.pack()
create_new_neural_network_button.pack()
#print_current_network_structure_button.pack()
train_network_button.pack()
predict_button.pack()
delete_network_button.pack()
#load_neural_network_button.pack()
save_neural_network_button.pack()
current_data_label.pack()
change_data_combobox.pack()
upload_data_button.pack()
delete_data_button.pack()
visual_frame.pack()
#print_data_button.pack()

In [20]:
#open tk window
main_window.mainloop()

In [21]:
print(current_neural_network.accuracy)

0.892469
